baseline.ipynb<br>
.. └ data<br>
.... ├ train.json<br>
.... ├ test.json<br>
.... └ sample_submission.csv<br>

# 사용 패키지

In [ ]:
# ### 서버(우분투)에 KoNLPy 설치하기: https://konlpy.org/ko/v0.5.2/install/

# ### 1. jdk 설치
# sudo apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl

# ### 2. konlpy 설치
# python3 -m pip install --upgrade pip
# python3 -m pip install konlpy

# ### 3. Mecab 설치
# sudo apt-get install curl git
# bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import random
import math
import copy
import gc
from tqdm import tqdm
from glob import glob
from konlpy.tag import Mecab

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action='ignore')

## 랜덤 시드 고정

In [19]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [20]:
seed_everything(42)

# 데이터 로드

In [21]:
DIR = "data/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

In [22]:
with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [23]:
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            # context += ' '
            context += '\n\n'  # 태깅할 때 보기 편하게 개행 2번
            
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            # context += ' '
            context += '\n\n'  # 태깅할 때 보기 편하게 개행 2번
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [24]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context

In [25]:
train.head()

,uid,title,region,context,summary,total
1000,1000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1001,1001,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
1002,1002,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
1003,1003,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
1004,1004,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


In [26]:
train['context'].iloc[462]

'의사일정 제1항, 2014년도 제2회 세입ㆍ세출 추가경정예산안 승인의 건, 의사일정 제2항, 2014년도 기금운용계획변경안 승인의 건을 일괄 상정합니다.\n\n예산결산특별위원회 위원장이신 한동완 위원장께서는 나오셔서 보고해 주시기 바랍니다.\n\n예산결산특별위원회위원장 한동완 의원입니다. 지난 12월 16일부터 12월 18일까지 3일간에 걸쳐 실시한 2014년도 제2회 세입ㆍ세출 추가경정예산안 및 기금운용계획변경안에 대한 심사 결과를 보고드리겠습니다.\n\n2014년도 제2회 세입ㆍ세출 추가경정예산안은 일반회계와 11개의 특별회계 및 기금운용계획변경안이 제출되어 본 위원을 포함한 일곱 분의 위원님과 부군수님, 실과소장 여러분이 참석한 가운데 진지하고 심도 있게 예산안을 심사하였다고 생각합니다.\n\n그럼 보고사항으로, 먼저 3페이지입니다. 본 위원회에서 심사한 2014년도 제2회 추가경정예산안 총 예산규모는 4,782억 6,724만 9천 원으로 일반회계 세입ㆍ세출예산액은 4,036억 6,377만 4천 원이며, 11개 특별회계 예산은 746억 347만 5천 원으로 편성되었으며, 3페이지부터 9페이지까지는 일반회계 및 특별회계, 기금운용은 유인물로 갈음하겠습니다.\n\n다음은 10페이지입니다. 지난 12월 16일 예산안 제출 후 변경된 사업 일부 추가에 따라 2014년도 제2회 세입ㆍ세출 추가경정예산안에 대한 수정예산안이 제출되어 본예산과 함께 심사하였으며, 기정예산액과 동일한 규모로 10페이지부터 15페이지까지 유인물을 참고하여 주시기 바랍니다.\n\n다음은 16페이지 전문위원 검토보고 요지는 생략하겠으며, 예산심사 결과에 대한 총평을 말씀드리겠습니다. 예산결산특별위원회에서는 2014년도 제2회 추가경정예산안에 대하여 충분한 설명을 듣고 요구된 예산의 효과성과 필요성, 국도비 보조사업 조정과 예산 편성상의 문제점을 다각도로 검토함으로써 예산 운용의 내실화를 위해 심도 있게 심사를 하였습니다. 심사 결과 제2회 추가경정예산안은 인건비, 공공요금 등 법정 및 경상

In [27]:
train['uid'].iloc[0]

1000

### uid와 context만 뽑아 태깅 문서로 저장 (화자 정보는 날아감) - train data

In [28]:
train_doc = pd.DataFrame()
train_doc['uid'] = train['uid']
train_doc['document_text'] = train['context']


save_file = "train_doc.csv"
save_dir = os.path.join(DIR, save_file)
train_doc.to_csv(save_dir, header=True, index=False)


print("train data 개수: ", len(train_doc))
train_doc.head()

train data 개수:  2994


,uid,document_text
1000,1000,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제...
1001,1001,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결...
1002,1002,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...
1003,1003,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하여...
1004,1004,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 정...


In [29]:
train.shape

(2994, 6)

### uid와 context만 뽑아 태깅 문서로 저장 (화자 정보는 날아감) - test data

In [30]:
test.head()

,uid,title,region,context,total
2000,2000,제235회 본회의 제1차(2012.06.21.),음성,의석을 정돈하여 주시기 바랍니다.\n\n성원이 되었으므로 지금부터 음성군의회 제23...,제235회 본회의 제1차(2012.06.21.) 음성 의석을 정돈하여 주시기 ...
2001,2001,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다.\...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제1항, 음성군..."
2002,2002,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다.\n\n제235회 제1차 ...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제2항, 회의록..."
2003,2003,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다.\n\n예산결산특별위원회...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제3항, 예산결..."
2004,2004,제235회 본회의 제1차(2012.06.21.),음성,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다.\n\n대표발...","제235회 본회의 제1차(2012.06.21.) 음성 의사일정 제4항, 환경분..."


In [31]:
test.shape

(506, 5)

In [32]:
test_doc = pd.DataFrame()
test_doc['uid'] = test['uid']
test_doc['document_text'] = test['context']


save_file = "test_doc.csv"
save_dir = os.path.join(DIR, save_file)
test_doc.to_csv(save_dir, header=True, index=False)


print("test data 개수: ", len(test_doc))
test_doc.head()

test data 개수:  506


,uid,document_text
2000,2000,의석을 정돈하여 주시기 바랍니다.\n\n성원이 되었으므로 지금부터 음성군의회 제23...
2001,2001,"의사일정 제1항, 음성군의회 제235회 제1차 정례회 회기결정의 건을 상정합니다.\..."
2002,2002,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다.\n\n제235회 제1차 ..."
2003,2003,"의사일정 제3항, 예산결산특별위원회 구성의 건을 상정합니다.\n\n예산결산특별위원회..."
2004,2004,"의사일정 제4항, 환경분야 현지확인 특별위원회 구성결의안을 상정합니다.\n\n대표발..."
